In [ ]:
import time
import logging
from logging.config import dictConfig
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, morphology, exposure, transform, color
from skimage.measure import label, regionprops
from random import randint
import sys


def main():
    filePhoto = "img/photo.jpg"
    fileMask = "img/mask.jpg"
    img_grayscale = color.rgb2gray(io.imread(filePhoto))
    img_mask = color.rgb2gray(cv2.imread(fileMask))
    img = inPainting(img_grayscale,img_mask)


if __name__ == '__main__':
    main()

In [ ]:
def findMatches(temp,convPatches, mask_final, mask_gaussian, windowSize, halfWindow, ErrThreshold):

    temp = np.reshape(temp, windowSize*windowSize)
    mask_gaussian = np.reshape(mask_gaussian, windowSize*windowSize)
    mask_final = np.reshape(mask_final, windowSize*windowSize)
    total_weight = np.sum(np.multiply(mask_gaussian, mask_final))
    distance = (convPatches-temp)**2
    ssd = np.sum((distance*mask_gaussian*mask_final) / total_weight, axis=1)
    min_error = min(ssd)
    j = int(((2 * halfWindow + 1) ** 2) / 2)
    return [[err, convPatches[i][j]] for i, err in enumerate(ssd) if err <= min_error*(1+ErrThreshold)]

In [ ]:
def remove_mask(img,mask):
  
    x = np.where(mask>0.9)
    for i in range(len(x[0])):
        img[x[0][i]][x[1][i]]=0
        mask[x[0][i]][x[1][i]]=1

def isEmpty(mask):
    return (0 == len(np.nonzero(mask)[0]))

def inPainting(img, mask):
    remove_mask(img,mask)
    mask = np.floor(mask)
    fileName = "temp.png"
    print("---------")
    print(img)
    print("---------")
    textureSynthesis(img,11)

def convolutionPatches_mod(img, filledMap, halfWindow):
    convPatches = []
    c = 0
    for i in range(halfWindow, img.shape[0] - halfWindow - 1):
        for j in range(halfWindow, img.shape[1] - halfWindow - 1):
            if 0 in filledMap[i - halfWindow:i + halfWindow + 1, j - halfWindow: j + halfWindow + 1]:
                c = c + 1
            else:
                convPatches.append(np.reshape(
                    img[i - halfWindow:i + halfWindow + 1, j - halfWindow: j + halfWindow + 1],
                    (2 * halfWindow + 1) ** 2))
    convPatches = np.double(convPatches)
    return convPatches

def textureSynthesis(img, windowSize):

    row, col = np.shape(img)
    ErrThreshold = 0.1
    MaxErrThreshold = 0.3
    sigma = windowSize/6.4
    seed = 3
    halfWindow = (windowSize - 1) // 2
    totalPixels = img.shape[0] * img.shape[1]
    print("---------")
    print(img)
    print("---------")
    filledMap = np.ceil(img)
    filledPixels = np.sum(filledMap)

    convPatches = convolutionPatches_mod(img, filledMap, halfWindow)
    print(convPatches.shape)
    synthesizedImage = img

    gaussMask = GaussMask(windowSize,sigma)

    synthImagePad = np.lib.pad(synthesizedImage, halfWindow, mode='constant', constant_values=0)
    filledMapPad = np.lib.pad(filledMap, halfWindow, mode='constant', constant_values=0)

    print(filledPixels)
    print(totalPixels)

    while filledPixels < totalPixels:
        progress = False
        pixelList = np.nonzero(morphology.binary_dilation(filledMap) - filledMap)
        neighbors = []
        neighbors.append([np.sum(filledMap[pixelList[0][i] - halfWindow : pixelList[0][i] + halfWindow + 1, pixelList[1][i] - halfWindow : pixelList[1][i] + halfWindow + 1]) for i in range(len(pixelList[0]))])
        decreasingOrder = np.argsort(-np.array(neighbors, dtype=int))

        for i in decreasingOrder[0]:
            temp = synthImagePad[pixelList[0][i] - halfWindow + halfWindow:pixelList[0][i] + halfWindow + halfWindow + 1, pixelList[1][i] - halfWindow + halfWindow:pixelList[1][i] + halfWindow + halfWindow + 1]
            validMask = filledMapPad[pixelList[0][i] - halfWindow + halfWindow:pixelList[0][i] + halfWindow + halfWindow + 1, pixelList[1][i] - halfWindow + halfWindow:pixelList[1][i] + halfWindow + halfWindow + 1]
            bestMatches = findMatches(temp,convPatches,validMask,gaussMask,windowSize, halfWindow, ErrThreshold)

            bestMatch = randint(0, len(bestMatches)-1)

            if bestMatches[bestMatch][0]<=MaxErrThreshold:
                synthImagePad[halfWindow+pixelList[0][i]][halfWindow+pixelList[1][i]] = bestMatches[bestMatch][1]
                synthesizedImage[pixelList[0][i]][pixelList[1][i]] = bestMatches[bestMatch][1]
                filledMapPad[halfWindow+pixelList[0][i]][halfWindow+pixelList[1][i]] = 1
                filledMap[pixelList[0][i]][pixelList[1][i]] = 1
                filledPixels+=1

                progress = True
        if not progress:
            MaxErrThreshold *= 1.1
        i = (filledPixels/totalPixels)*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush()
    io.imsave("synthesizedImage.png", synthesizedImage)
    plt.show()
    return


def GaussMask(windowSize, sigma):
    x, y = np.mgrid[-windowSize//2 + 1:windowSize//2 + 1, -windowSize//2 + 1:windowSize//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()